## Regressione Lineare e Polinomiale per il Prezzo delle Zucche - Lezione 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografica di Dasani Madipalli</figcaption>


<!--![Infografica di Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.it.png){width="800"}-->

#### Introduzione

Finora hai esplorato cosa sia la regressione utilizzando dati di esempio raccolti dal dataset sui prezzi delle zucche che useremo per tutta questa lezione. Hai anche visualizzato i dati con `ggplot2`. 💪

Ora sei pronto per approfondire la regressione per il Machine Learning. In questa lezione, imparerai di più su due tipi di regressione: *regressione lineare di base* e *regressione polinomiale*, insieme ad alcune nozioni matematiche alla base di queste tecniche.

> In tutto questo curriculum, assumiamo una conoscenza minima della matematica e cerchiamo di renderla accessibile agli studenti provenienti da altri campi. Fai attenzione a note, 🧮 richiami, diagrammi e altri strumenti di apprendimento per facilitare la comprensione.

#### Preparazione

Come promemoria, stai caricando questi dati per porre loro delle domande.

-   Qual è il momento migliore per comprare le zucche?

-   Quale prezzo posso aspettarmi per una cassa di zucche in miniatura?

-   Dovrei comprarle in cesti da mezzo bushel o in scatole da 1 1/9 di bushel? Continuiamo a scavare in questi dati.

Nella lezione precedente, hai creato un `tibble` (una moderna reinterpretazione del data frame) e lo hai popolato con una parte del dataset originale, standardizzando i prezzi per bushel. Facendo ciò, tuttavia, sei riuscito a raccogliere solo circa 400 punti dati e solo per i mesi autunnali. Forse possiamo ottenere un po' più di dettagli sulla natura dei dati pulendoli ulteriormente? Vedremo... 🕵️‍♀️

Per questo compito, avremo bisogno dei seguenti pacchetti:

-   `tidyverse`: Il [tidyverse](https://www.tidyverse.org/) è una [collezione di pacchetti R](https://www.tidyverse.org/packages) progettata per rendere la scienza dei dati più veloce, facile e divertente!

-   `tidymodels`: Il framework [tidymodels](https://www.tidymodels.org/) è una [collezione di pacchetti](https://www.tidymodels.org/packages/) per la modellazione e il machine learning.

-   `janitor`: Il pacchetto [janitor](https://github.com/sfirke/janitor) fornisce semplici strumenti per esaminare e pulire dati disordinati.

-   `corrplot`: Il pacchetto [corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) offre uno strumento visivo esplorativo per la matrice di correlazione, supportando il riordino automatico delle variabili per aiutare a rilevare schemi nascosti tra le variabili.

Puoi installarli con il seguente comando:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Lo script qui sotto verifica se hai i pacchetti necessari per completare questo modulo e li installa per te nel caso in cui manchino.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Caricheremo successivamente questi fantastici pacchetti e li renderemo disponibili nella nostra sessione R corrente. (Questo è solo a scopo illustrativo, `pacman::p_load()` lo ha già fatto per te)

## 1. Una linea di regressione lineare

Come hai imparato nella Lezione 1, l'obiettivo di un esercizio di regressione lineare è essere in grado di tracciare una *linea* *di* *miglior adattamento* per:

-   **Mostrare le relazioni tra variabili**. Mostrare la relazione tra le variabili.

-   **Fare previsioni**. Fare previsioni accurate su dove un nuovo punto dati potrebbe cadere in relazione a quella linea.

Per disegnare questo tipo di linea, utilizziamo una tecnica statistica chiamata **Regressione dei Minimi Quadrati**. Il termine `minimi quadrati` significa che tutti i punti dati che circondano la linea di regressione vengono elevati al quadrato e poi sommati. Idealmente, quella somma finale è il più piccola possibile, perché vogliamo un numero basso di errori, o `minimi quadrati`. Pertanto, la linea di miglior adattamento è la linea che ci dà il valore più basso per la somma degli errori al quadrato - da qui il nome *regressione dei minimi quadrati*.

Facciamo questo perché vogliamo modellare una linea che abbia la minima distanza cumulativa da tutti i nostri punti dati. Inoltre, eleviamo al quadrato i termini prima di sommarli poiché ci interessa la loro grandezza piuttosto che la loro direzione.

> **🧮 Mostrami la matematica**
>
> Questa linea, chiamata *linea di miglior adattamento*, può essere espressa da [un'equazione](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` è la '`variabile esplicativa` o `predittore`'. `Y` è la '`variabile dipendente` o `risultato`'. La pendenza della linea è `b` e `a` è l'intercetta sull'asse y, che si riferisce al valore di `Y` quando `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "pendenza = $y/x$")
    Infografica di Jen Looper
>
> Per prima cosa, calcola la pendenza `b`.
>
> In altre parole, e riferendoci alla domanda originale sui dati delle zucche: "prevedere il prezzo di una zucca per bushel in base al mese", `X` si riferirebbe al prezzo e `Y` si riferirebbe al mese di vendita.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.it.png)
    Infografica di Jen Looper
> 
> Calcola il valore di Y. Se stai pagando circa \$4, deve essere aprile!
>
> La matematica che calcola la linea deve dimostrare la pendenza della linea, che dipende anche dall'intercetta, ovvero dove si trova `Y` quando `X = 0`.
>
> Puoi osservare il metodo di calcolo per questi valori sul sito web [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Visita anche [questo calcolatore dei minimi quadrati](https://www.mathsisfun.com/data/least-squares-calculator.html) per vedere come i valori numerici influenzano la linea.

Non così spaventoso, vero? 🤓

#### Correlazione

Un altro termine da comprendere è il **Coefficiente di Correlazione** tra le variabili X e Y date. Utilizzando un diagramma a dispersione, puoi visualizzare rapidamente questo coefficiente. Un grafico con punti dati distribuiti in una linea ordinata ha alta correlazione, mentre un grafico con punti dati distribuiti ovunque tra X e Y ha bassa correlazione.

Un buon modello di regressione lineare sarà quello che ha un alto Coefficiente di Correlazione (più vicino a 1 che a 0) utilizzando il metodo della Regressione dei Minimi Quadrati con una linea di regressione.


## **2. Una danza con i dati: creare un data frame che verrà utilizzato per la modellazione**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Opera d'arte di @allison_horst</figcaption>


<!--![Opera d'arte di \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.it.jpg){width="700"}-->


Carica le librerie necessarie e il dataset. Converti i dati in un data frame contenente un sottoinsieme dei dati:

-   Considera solo le zucche con prezzo al bushel

-   Converti la data in un mese

-   Calcola il prezzo come media tra il prezzo massimo e minimo

-   Converti il prezzo per riflettere la quantità al bushel

> Abbiamo trattato questi passaggi nella [lezione precedente](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Nello spirito della pura avventura, esploriamo il [`pacchetto janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) che fornisce funzioni semplici per esaminare e pulire dati sporchi. Ad esempio, diamo un'occhiata ai nomi delle colonne dei nostri dati:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Possiamo fare meglio. Convertiamo questi nomi di colonna in `friendR` utilizzando la convenzione [snake_case](https://en.wikipedia.org/wiki/Snake_case) con `janitor::clean_names`. Per saperne di più su questa funzione: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Molto tidyR 🧹! Ora, un ballo con i dati usando `dplyr` come nella lezione precedente! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Ottimo lavoro!👌 Ora hai un set di dati pulito e ordinato su cui puoi costruire il tuo nuovo modello di regressione!

Che ne dici di un grafico a dispersione?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Un grafico a dispersione ci ricorda che abbiamo dati mensili solo da agosto a dicembre. Probabilmente abbiamo bisogno di più dati per poter trarre conclusioni in modo lineare.

Diamo un'occhiata di nuovo ai nostri dati di modellazione:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

E se volessimo prevedere il `prezzo` di una zucca basandoci sulle colonne `città` o `pacchetto`, che sono di tipo carattere? Oppure, ancora più semplicemente, come potremmo trovare la correlazione (che richiede che entrambi i suoi input siano numerici) tra, ad esempio, `pacchetto` e `prezzo`? 🤷🤷

I modelli di machine learning funzionano meglio con caratteristiche numeriche piuttosto che con valori testuali, quindi in genere è necessario convertire le caratteristiche categoriche in rappresentazioni numeriche.

Questo significa che dobbiamo trovare un modo per riformattare i nostri predittori per renderli più facili da utilizzare efficacemente da un modello, un processo noto come `feature engineering`.


## 3. Pre-elaborazione dei dati per la modellazione con recipes 👩‍🍳👨‍🍳

Le attività che riformattano i valori dei predittori per renderli più facili da utilizzare efficacemente da un modello sono chiamate `feature engineering`.

Modelli diversi hanno requisiti di pre-elaborazione differenti. Ad esempio, i minimi quadrati richiedono `l'encoding delle variabili categoriche` come mese, varietà e city_name. Questo implica semplicemente `tradurre` una colonna con `valori categorici` in una o più `colonne numeriche` che sostituiscono l'originale.

Ad esempio, supponiamo che i tuoi dati includano la seguente caratteristica categorica:

|  city   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

Puoi applicare l'*ordinal encoding* per sostituire ogni categoria con un valore intero unico, come questo:

| city |
|:----:|
|  0   |
|  1   |
|  2   |

Ed è proprio quello che faremo con i nostri dati!

In questa sezione, esploreremo un altro fantastico pacchetto di Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - progettato per aiutarti a pre-elaborare i tuoi dati **prima** di addestrare il tuo modello. Alla base, una recipe è un oggetto che definisce quali passaggi devono essere applicati a un set di dati per prepararlo alla modellazione.

Ora, creiamo una recipe che prepara i nostri dati per la modellazione sostituendo un intero unico per tutte le osservazioni nelle colonne dei predittori:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Fantastico! 👏 Abbiamo appena creato la nostra prima ricetta che specifica un risultato (prezzo) e i suoi corrispondenti predittori, e che tutte le colonne dei predittori devono essere codificate in un set di numeri interi 🙌! Analizziamola rapidamente:

-   La chiamata a `recipe()` con una formula indica alla ricetta i *ruoli* delle variabili utilizzando i dati di `new_pumpkins` come riferimento. Ad esempio, la colonna `price` è stata assegnata al ruolo di `outcome`, mentre il resto delle colonne è stato assegnato al ruolo di `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` specifica che tutti i predittori devono essere convertiti in un set di numeri interi, con la numerazione che parte da 0.

Siamo sicuri che potresti avere pensieri del tipo: "È fantastico!! Ma cosa succede se avessi bisogno di confermare che le ricette stanno facendo esattamente ciò che mi aspetto? 🤔"

È un pensiero eccellente! Vedi, una volta definita la tua ricetta, puoi stimare i parametri necessari per preprocessare effettivamente i dati e poi estrarre i dati elaborati. Non è qualcosa che fai tipicamente quando usi Tidymodels (vedremo la convenzione normale tra poco-\> `workflows`), ma può essere utile quando vuoi fare un controllo di coerenza per confermare che le ricette stiano facendo ciò che ti aspetti.

Per questo, avrai bisogno di due verbi aggiuntivi: `prep()` e `bake()`, e come sempre, i nostri piccoli amici di R di [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) ti aiutano a capire meglio!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): stima i parametri necessari da un set di addestramento che possono essere successivamente applicati ad altri set di dati. Ad esempio, per una determinata colonna predittore, quale osservazione verrà assegnata all'intero 0, 1, 2, ecc.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): prende una ricetta preparata e applica le operazioni a qualsiasi set di dati.

Detto ciò, prepariamo e applichiamo le nostre ricette per confermare davvero che, dietro le quinte, le colonne predittore saranno prima codificate prima che un modello venga adattato.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Evviva! 🥳 I dati elaborati `baked_pumpkins` hanno tutti i loro predittori codificati, confermando che i passaggi di preprocessing definiti come la nostra ricetta funzioneranno come previsto. Questo li rende più difficili da leggere per te, ma molto più comprensibili per Tidymodels! Prenditi un po' di tempo per scoprire quale osservazione è stata mappata a un intero corrispondente.

Vale anche la pena menzionare che `baked_pumpkins` è un data frame su cui possiamo eseguire calcoli.

Ad esempio, proviamo a trovare una buona correlazione tra due punti dei tuoi dati per potenzialmente costruire un buon modello predittivo. Useremo la funzione `cor()` per farlo. Digita `?cor()` per scoprire di più sulla funzione.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


A quanto pare, c'è solo una debole correlazione tra la Città e il Prezzo. Tuttavia, c'è una correlazione un po' migliore tra il Pacchetto e il suo Prezzo. Ha senso, vero? Normalmente, più grande è la scatola di prodotti, più alto è il prezzo.

Già che ci siamo, proviamo anche a visualizzare una matrice di correlazione di tutte le colonne utilizzando il pacchetto `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Molto meglio.

Una buona domanda da porre ora a questi dati potrebbe essere: '`Quale prezzo posso aspettarmi per un determinato pacchetto di zucche?`' Andiamo subito al dunque!

> Nota: Quando **`bake()`** la ricetta preparata **`pumpkins_prep`** con **`new_data = NULL`**, estrai i dati di addestramento elaborati (cioè codificati). Se avessi un altro set di dati, ad esempio un set di test, e volessi vedere come una ricetta lo pre-processerebbe, ti basterebbe fare il bake di **`pumpkins_prep`** con **`new_data = test_set`**.

## 4. Costruire un modello di regressione lineare

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografica di Dasani Madipalli</figcaption>


Ora che abbiamo creato una ricetta e confermato che i dati verranno pre-processati correttamente, passiamo a costruire un modello di regressione per rispondere alla domanda: `Quale prezzo posso aspettarmi per un determinato pacchetto di zucca?`

#### Addestrare un modello di regressione lineare utilizzando il set di addestramento

Come avrai già intuito, la colonna *price* è la variabile `outcome`, mentre la colonna *package* è la variabile `predictor`.

Per fare ciò, inizieremo dividendo i dati in modo che l'80% vada nel set di addestramento e il 20% nel set di test, poi definiremo una ricetta che codificherà la colonna predictor in un insieme di interi, quindi costruiremo una specifica del modello. Non prepareremo e non cuoceremo la nostra ricetta, poiché sappiamo già che pre-processerà i dati come previsto.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Ottimo lavoro! Ora che abbiamo una ricetta e una specifica del modello, dobbiamo trovare un modo per unirle in un oggetto che prima pre-elabori i dati (prep+bake dietro le quinte), adatti il modello ai dati pre-elaborati e consenta anche eventuali attività di post-elaborazione. Che ne dici per la tua tranquillità!🤩

In Tidymodels, questo pratico oggetto si chiama [`workflow`](https://workflows.tidymodels.org/) e contiene comodamente i tuoi componenti di modellazione! Questo è ciò che chiameremmo *pipeline* in *Python*.

Quindi, mettiamo tutto insieme in un workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Inoltre, un workflow può essere adattato/allenato nello stesso modo in cui può esserlo un modello.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Dai coefficienti ottenuti durante l'addestramento, possiamo osservare i valori appresi. Essi rappresentano i coefficienti della retta di miglior adattamento che ci fornisce l'errore complessivo più basso tra la variabile reale e quella predetta.

#### Valutare le prestazioni del modello utilizzando il set di test

È il momento di vedere come si è comportato il modello 📏! Come possiamo farlo?

Ora che abbiamo addestrato il modello, possiamo utilizzarlo per fare previsioni sul `test_set` usando `parsnip::predict()`. Successivamente, possiamo confrontare queste previsioni con i valori effettivi delle etichette per valutare quanto bene (o meno!) il modello stia funzionando.

Iniziamo facendo previsioni per il set di test e poi uniamo le colonne al set di test.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Sì, hai appena addestrato un modello e lo hai utilizzato per fare previsioni! 🔮 È valido? Valutiamo le prestazioni del modello!

In Tidymodels, lo facciamo utilizzando `yardstick::metrics()`! Per la regressione lineare, concentriamoci sui seguenti indicatori:

-   `Root Mean Square Error (RMSE)`: La radice quadrata del [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Questo fornisce una metrica assoluta nella stessa unità dell'etichetta (in questo caso, il prezzo di una zucca). Più piccolo è il valore, migliore è il modello (in termini semplici, rappresenta il prezzo medio di errore delle previsioni!).

-   `Coefficient of Determination (di solito noto come R-squared o R2)`: Una metrica relativa in cui un valore più alto indica una migliore aderenza del modello. In sostanza, questa metrica rappresenta quanto della varianza tra i valori previsti e quelli reali delle etichette il modello è in grado di spiegare.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Ecco che la performance del modello cala. Vediamo se possiamo ottenere un'indicazione migliore visualizzando un grafico a dispersione del pacchetto e del prezzo, quindi utilizzando le previsioni fatte per sovrapporre una linea di miglior adattamento.

Questo significa che dovremo preparare e trasformare il set di test per codificare la colonna del pacchetto, quindi unire questo alle previsioni fatte dal nostro modello.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Fantastico! Come puoi vedere, il modello di regressione lineare non generalizza molto bene la relazione tra un pacchetto e il suo prezzo corrispondente.

🎃 Congratulazioni, hai appena creato un modello che può aiutarti a prevedere il prezzo di alcune varietà di zucche. Il tuo campo di zucche per le festività sarà splendido. Ma probabilmente puoi creare un modello migliore!

## 5. Costruisci un modello di regressione polinomiale

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografica di Dasani Madipalli</figcaption>


<!--![Infografica di Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.it.png){width="800"}-->


A volte i nostri dati potrebbero non avere una relazione lineare, ma vogliamo comunque prevedere un risultato. La regressione polinomiale può aiutarci a fare previsioni per relazioni non lineari più complesse.

Prendiamo, ad esempio, la relazione tra il pacchetto e il prezzo nel nostro set di dati sulle zucche. Sebbene a volte ci sia una relazione lineare tra le variabili - più grande è la zucca in volume, più alto è il prezzo - a volte queste relazioni non possono essere rappresentate come un piano o una linea retta.

> ✅ Ecco [alcuni altri esempi](https://online.stat.psu.edu/stat501/lesson/9/9.8) di dati che potrebbero utilizzare la regressione polinomiale
>
> Dai un'altra occhiata alla relazione tra Varietà e Prezzo nel grafico precedente. Questo scatterplot sembra necessariamente analizzabile con una linea retta? Forse no. In questo caso, puoi provare la regressione polinomiale.
>
> ✅ I polinomi sono espressioni matematiche che possono consistere in una o più variabili e coefficienti

#### Addestrare un modello di regressione polinomiale utilizzando il set di addestramento

La regressione polinomiale crea una *linea curva* per adattarsi meglio ai dati non lineari.

Vediamo se un modello polinomiale offrirà prestazioni migliori nel fare previsioni. Seguiremo una procedura abbastanza simile a quella che abbiamo seguito in precedenza:

-   Creare una ricetta che specifichi i passaggi di pre-elaborazione da eseguire sui nostri dati per prepararli alla modellazione, ad esempio: codificare i predittori e calcolare i polinomi di grado *n*

-   Costruire una specifica del modello

-   Combinare la ricetta e la specifica del modello in un flusso di lavoro

-   Creare un modello adattando il flusso di lavoro

-   Valutare quanto bene il modello si comporta sui dati di test

Iniziamo subito!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Valutare le prestazioni del modello

👏👏Hai creato un modello polinomiale, ora facciamo delle previsioni sul set di test!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, valutiamo come il modello ha performato sul test_set usando `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Prestazioni molto migliori.

L'`rmse` è diminuito da circa 7 a circa 3, un'indicazione di un errore ridotto tra il prezzo reale e il prezzo previsto. Puoi *liberamente* interpretarlo come se, in media, le previsioni errate sbagliassero di circa 3$. L'`rsq` è aumentato da circa 0,4 a 0,8.

Tutti questi parametri indicano che il modello polinomiale funziona decisamente meglio rispetto al modello lineare. Ottimo lavoro!

Vediamo se possiamo visualizzarlo!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Puoi vedere una linea curva che si adatta meglio ai tuoi dati! 🤩

Puoi renderla ancora più fluida passando una formula polinomiale a `geom_smooth` in questo modo:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Proprio come una curva fluida!🤩

Ecco come fare una nuova previsione:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


La previsione del `modello polinomiale` ha senso, considerando i grafici a dispersione di `price` e `package`! E, se questo è un modello migliore rispetto al precedente, osservando gli stessi dati, dovrai pianificare un budget per queste zucche più costose!

🏆 Ben fatto! Hai creato due modelli di regressione in una sola lezione. Nella sezione finale sulla regressione, imparerai la regressione logistica per determinare le categorie.

## **🚀Sfida**

Prova diverse variabili in questo notebook per vedere come la correlazione corrisponde alla precisione del modello.

## [**Quiz post-lezione**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Revisione & Studio Autonomo**

In questa lezione abbiamo imparato la Regressione Lineare. Esistono altri tipi importanti di Regressione. Leggi le tecniche Stepwise, Ridge, Lasso ed Elasticnet. Un buon corso per approfondire è il [corso di Stanford Statistical Learning](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Se vuoi imparare di più su come utilizzare il fantastico framework Tidymodels, consulta le seguenti risorse:

-   Sito web di Tidymodels: [Inizia con Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn e Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **GRAZIE A:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) per aver creato le incredibili illustrazioni che rendono R più accogliente e coinvolgente. Trova altre illustrazioni nella sua [galleria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
